<a href="https://colab.research.google.com/github/bcwalraven/TD3/blob/master/TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybullet

     |████████████████████████████████| 48.1MB 1.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/c3/92/94/4f9c3e1b1769fc834a102b7ed04cc07ebad8663bfe11e2d830
Successfully built pybullet


In [0]:
#importing the libraries
[ ]
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## **Step 1: We initialize the Experience Replay memory**

In [0]:
class ReplayBuffer(object):
  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0
    
  def add(self, transition):
    if len(self.storage == self.max_size):
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)
      
  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storate), batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind:
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(actions, copy=False))
      batch_rewards.append(np.array(reward, copy=False))    
      batch_dones.append(np.array(done, copy=False))
      
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1) 

## Step 2: We build one neural network for the Actor model and one neural network for the Actor target

In [0]:
class Actor(nn.Module):
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action
    
  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

## Step 3: We build two neural networks for the two Critic models and two neural networks for the two Critic targets

In [0]:
class Critic(nn.Module):
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    #defining the first critic network
    self.layer_1_1 = nn.Linear(state_dim + sction_dim, 400)
    self.layer_2_2 = nn.Linear(400, 300)
    self.layer_3_3 = nn.Linear(300, 1)
    #defining the second critic network
    self.layer_2_1 = nn.Linear(state_dim + sction_dim, 400)
    self.layer_2_2 = nn.Linear(400, 300)
    self.layer_2_3 = nn.Linear(300, 1)
    
  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # forward prop through the first critic network
    x1 = F.relu(self.layer_1_1(xu))
    x1 = F.relu(self.layer_1_2(x1))
    x1 = self.layer_1_3(x1)
    # forward prop through the second critic network 
    x2 = F.relu(self.layer_2_1(xu))
    x2 = F.relu(self.layer_2_2(x2))
    x2 = self.layer_2_3(x2)
    return x1, x2
  
  def Q_1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1_1(xu))
    x1 = F.relu(self.layer_1_2(x1))
    x1 = self.layer_1_3(x1)
    return x1

In [0]:
# selecting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# building the whole training process into a class

class TD3(object):
  def __init__(self, state_dim, action_dim, max_action):
    # initilzing the actor models
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    # initilizing the critic models
    self.critic = Critic(self, state_dim, action_dim).to(device)
    self.critic_target = Critic(self, state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic_state_dict())
    self.actor_optimizer = torch.optim.Adam(self.critic.parameters())
    # specifying max action
    self.max_action = max_action